# important details
##### Set shuffle equals true or model cant seem to learn (local maxima)

###### Key details
1. Tensorflow 1.9.0
2. Keras 2.2.0

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential, load_model, Model
from keras.layers import Dropout, Flatten, Dense, Input, AveragePooling2D
from keras import applications
from keras.applications.inception_v3 import InceptionV3, preprocess_input 
from keras.utils.np_utils import to_categorical
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam  
import keras,os,re,math,itertools
import matplotlib.pyplot as plt
import keras.backend as K
from ipywidgets import interact, interactive, fixed
from sklearn.metrics import confusion_matrix

#import cv2
#%load_ext memory_profiler


#check GPU usage
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# dimensions of our images.
img_width, img_height = 299, 299

#important PATH variables
rootPath = r'/notebooks/storage'
full_model_weights_path = os.path.join(rootPath,'weights','full_model_weights.h5')
trainUrl = os.path.join(rootPath, 'data', 'Train')
testUrl =  os.path.join(rootPath, 'data', 'Test')
valUrl =  os.path.join(rootPath, 'data', 'Val')
class_indices_path = os.path.join(rootPath,'class_indices.npy')
modelCheckpointPath = os.path.join(rootPath,'models','model4b.{epoch:02d}-{val_loss:.2f}.hdf5')
csvLoggerPath = os.path.join(rootPath,'log','model4b.log')
            

# number of epochs to train top model
epochs = 20

# batch size used by flow_from_directory and predict_generator, try 128 if possible
batch_size = 64
#important hyper-parameters
metrics = ['accuracy']

datagenTrain = ImageDataGenerator(
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False, # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2,
    fill_mode='nearest',
    preprocessing_function = preprocess_input) #preprocess_input is the preprocessing function used in 
                                                #original inceptionV3 model

generatorTrain = datagenTrain.flow_from_directory(
    trainUrl,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

#repeat the prediction for validation dataset
datagenVal = ImageDataGenerator(preprocessing_function = preprocess_input)

generatorVal = datagenVal.flow_from_directory(
    valUrl,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle= True)



#repeat the prediction for test dataset
datagenTest = ImageDataGenerator(preprocessing_function = preprocess_input)

generatorTest = datagenVal.flow_from_directory(
    testUrl,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle= True)

#calculate some useful variables
nb_train_samples = len(generatorTrain.filenames)
num_classes = len(generatorTrain.class_indices)
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
np.save(class_indices_path, generatorTrain.class_indices)    # save the class indices to use use later in predictions
    
#predict the number of steps required for batch size
nb_validation_samples = len(generatorVal.filenames)
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17321941593608493447
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11270533940
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4626270998032556798
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
Found 276675 images belonging to 406 classes.
Found 34471 images belonging to 406 classes.
Found 34478 images belonging to 406 classes.


In [2]:
#Helper functions
def show_images(unprocess=True):
#used to visualise images 
    plt.clf()
    def reverse_preprocess_input(x0):
        x = x0 / 2.0
        x += 0.5
        x *= 255.
        return x
    #import pdb;pdb.set_trace()
    for x in generatorTrain:
        fig, axes = plt.subplots(nrows=8, ncols=4)
        fig.set_size_inches(8, 8)
        page = 0
        page_size = 32
        start_i = page * page_size
        for i, ax in enumerate(axes.flat):
            img = x[0][i+start_i]
            
            if unprocess:
                im = ax.imshow(reverse_preprocess_input(img).astype('uint8') )
            else:
                im = ax.imshow((img/2.0)+.5) #matplotlib can only plot between range of 0 and 1
                
            ax.set_axis_off()
            ax.title.set_visible(False)
            ax.xaxis.set_ticks([])
            ax.yaxis.set_ticks([])
            for spine in ax.spines.values():
                spine.set_visible(False)

        plt.subplots_adjust(left=0, wspace=0, hspace=0)
        plt.show()
        break

def _build_model():
    #import pdb;pdb.set_trace()
    K.clear_session()
    base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
    x = base_model.output
    x = AveragePooling2D(pool_size=(8, 8))(x)
    x = Dropout(.4)(x)
    x = Flatten()(x)
    predictions = Dense(num_classes, init='glorot_uniform', W_regularizer=l2(.0005), activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

def _load_model(pathName= None):
    
    if pathName != None:
        print('loading model from {0}'.format(pathName))
        return keras.models.load_model(pathName)
    files  = os.listdir(os.path.join(rootPath,'models'))
    files.remove('.ipynb_checkpoints')
    lowest_val_loss = None
    model_path = None
    
    if len(files) == 0:
        print('There is no save models. Going to create a new model now')
        final_model = _build_model()
        return final_model
    
    #loop through all files and find the path of the file with lowest loss
    for file in files:
        file_val_loss = re.findall(r'\d\.\d{2}',file) #regex expression to extract the val loss
        if lowest_val_loss  == None:
            lowest_val_loss = file_val_loss
            model_path = file
        if file_val_loss < lowest_val_loss:
            lowest_val_loss = file_val_loss
            model_path = file
    print('Model loaded from path:{0}'.format(model_path))
    
    return keras.models.load_model(os.path.join(rootPath,'models',model_path))


def plot_graphs(history):
    # A history object is a keras object used to store accuracy and loss
    
    plt.figure(1)
    # summarize history for accuracy
    plt.subplot(211)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    # summarize history for loss
    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    

def train_top_model(pathName= None):
    #we are only training the top layer of the model and not the whole model
    model = _load_model(pathName)
    for layer in model.layers[:-4]:
        layer.trainable = False
    #import pdb;pdb.set_trace()
    adam =Adam(lr=0.0001)
    model.compile(optimizer= adam ,loss='categorical_crossentropy', metrics=metrics)
    history = model.fit_generator(generatorTrain,
                    validation_data=generatorVal,
                    steps_per_epoch=predict_size_train,
                    epochs=epochs,
                    verbose=1, max_queue_size=15, workers = 4, use_multiprocessing = True)
    model.save(os.path.join(rootPath,'models','model4b.00-9.99hdf5'))
    print('Model saved (pretrain model)')
    return history, model

def train_whole_model(pathName= None):
    
    #
    
    #import pdb;pdb.set_trace()
    model = _load_model(pathName)
    
    
    #only uncomment this code if training model from saved model which is only trained from train_top_model
    #for layer in model.layers:
    #    layer.trainable = True
    #model.compile(optimizer= 'adam',loss='categorical_crossentropy', metrics=metrics)
    
    csv_logger = CSVLogger(csvLoggerPath)
    checkpointer = ModelCheckpoint(filepath=modelCheckpointPath, verbose=1, save_best_only=True)   
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.0001,verbose =1)
    history = model.fit_generator(generatorTrain,
                    validation_data=generatorVal,
                    steps_per_epoch=predict_size_train,
                    epochs=epochs,
                    verbose=1,
                    callbacks=([csv_logger, checkpointer,reduce_lr]),
                    max_queue_size=10, workers = 8, 
                    use_multiprocessing = True)                
    return history, model

def model_evaluate(model):
    (eval_loss, eval_accuracy) = model.evaluate_generator(generatorTest,steps=None,verbose=1)
    print('Evaluating the:'.format(model.metrics_names))

    print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
    print("[INFO] Loss: {}".format(eval_loss))

# Training the model
### Training is split into 2 phases

##### Visualise some images to explore data

#### Phase one: Train top model
##### Not that effective

#train top model
history,model = train_top_model()

plot_graphs(history)

#### Phase two: Train the whole model

In [ ]:
#train whole model
final_history, final_model = train_whole_model()
plot_graphs(final_history)

# Evaluating the model

In [ ]:
final_model = _load_model()
model_evaluate(final_model)

In [ ]:
def predict(model,imgPath,topNum = 5):
    # load the class_indices saved in the earlier step
    class_dictionary = np.load('class_indices.npy').item()
    num_classes = len(class_dictionary)
    # add the path to your test image below
    orig = cv2.imread(imagePath)

    print("[INFO] loading and preprocessing image...")
    image = load_img(imagePath, target_size=(299, 299))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    #do all the preprocessing for the images
    # important! otherwise the predictions will be '0'
    image = preprocess_input(image)

    #getting the predictions
    y_pred_prob = model.predict(image)
    class_predicted_ix = np.argmax(y_pred_prob, axis=1)
    top_n_preds_ix = np.argpartition(y_pred_prob, -topNum)[:,-topNum:]

    #convert from ix to class label
    inv_map = {v: k for k, v in class_dictionary.items()}
    label = inv_map[inID]

    # get the prediction label
    print("Image ID: {}, Label: {}".format(class_predicted_ix[0], label))

    # display the predictions with the image
    cv2.putText(orig, "Predicted: {}".format(label), (10, 30),
                cv2.FONT_HERSHEY_PLAIN, 1.5, (43, 99, 255), 2)

    cv2.imshow("Classification", orig)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Plot the confusion matrix

In [3]:
def plot_confusion_matrix(model,generator):
    y_pred = model.predict_generator(generator,verbose=1)
    y_pred_value = np.argmax(y_pred, axis=1)
    conf = confusion_matrix(generatorTest.classes,y_pred_value)
    
    #find the largest n indices in a multi dimensional array
    def largest_indices(ary, n):
        """Returns the n largest indices from a numpy array."""
        flat = ary.flatten()
        indices = np.argpartition(flat, -n)[-n:]
        indices = indices[np.argsort(-flat[indices])]
        return np.unravel_index(indices, ary.shape)
    
    class_dictionary = np.load('/storage/class_indices.npy').item()
    inv_map = {v: k for k, v in class_dictionary.items()}
    row_ix ,col_ix = largest_indices(conf,50)

    #change from index value to labels
    row = [inv_map[x] for x in row_ix]
    col = [inv_map[x] for x in col_ix]

    return conf,list(zip(row,col))

Model loaded from path:model4b.05-1.27.hdf5
Found 34478 images belonging to 406 classes.
539/539 [==============================] - 392s 727ms/step
